In [1]:
import tensorflow as tf
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

/Users/didi/miniconda3/envs/tfenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/didi/miniconda3/envs/tfenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/didi/miniconda3/envs/tfenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/didi/miniconda3/envs/tfenv/lib/python3.6/s

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [9]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 100
num_classes = 10
display_step = 1
feature_size = 28 * 28

In [30]:

tf.reset_default_graph()  # 重置网络结构
# tf Graph Input
# mnist data image of shape 28*28=784
input_x = tf.placeholder(dtype=tf.float32, shape=[None, 784], name="input_x")
# 0-9 digits recognition => 10 classes
input_y = tf.placeholder(dtype=tf.float32, shape=[None, 10], name="input_y")
# ------ v1 不调用接口 ------
# Set model weights
W = tf.Variable(tf.zeros([feature_size, num_classes]))  # shape=(784, 10)
b = tf.Variable(tf.zeros([num_classes]))  # shape=(10,)
# Construct model
pred = tf.nn.softmax(tf.matmul(input_x, W) + b)  # Softmax, shape=(?, 10)
# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(input_y *
                      tf.log(pred), reduction_indices=1))

# # ------ v2 调用接口 ------
# logits = tf.layers.dense(inputs=input_x, units=num_classes, use_bias=True,
#                          kernel_initializer=tf.zeros_initializer, name="LR")  # shape=(?, 10)
# pred = tf.nn.softmax(logits=logits, name="pred")  # shape=(?, 10)
# cost = tf.losses.softmax_cross_entropy(onehot_labels=input_y, logits=logits)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(input_y, 1))
# Calculate accuracy for 3000 examples
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [31]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={input_x: batch_xs,
                                                          input_y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    
    print("Accuracy:", accuracy.eval(
        {input_x: mnist.test.images[:3000], input_y: mnist.test.labels[:3000]}))


Epoch: 0001 cost= 1.184468955
Epoch: 0002 cost= 0.665398103
Epoch: 0003 cost= 0.553089497
Epoch: 0004 cost= 0.498226045
Epoch: 0005 cost= 0.465735922
Epoch: 0006 cost= 0.442293742
Epoch: 0007 cost= 0.425786554
Epoch: 0008 cost= 0.411700056
Epoch: 0009 cost= 0.401514078
Epoch: 0010 cost= 0.392351863
Epoch: 0011 cost= 0.384608420
Epoch: 0012 cost= 0.378404692
Epoch: 0013 cost= 0.372837543
Epoch: 0014 cost= 0.366792649
Epoch: 0015 cost= 0.362560155
Epoch: 0016 cost= 0.358713187
Epoch: 0017 cost= 0.355256095
Epoch: 0018 cost= 0.351296299
Epoch: 0019 cost= 0.348659648
Epoch: 0020 cost= 0.345024994
Optimization Finished!
Accuracy: 0.88533336
